In [ ]:
!pip install demucs torchaudio
!pip uninstall -y soundfile
!pip install soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 4.2 MB/s eta 0:00:00
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.wh

Found existing installation: soundfile 0.12.1
Uninstalling soundfile-0.12.1:
ERROR: Operation cancelled by user


In [ ]:
import os
import zipfile
import torch
import torchaudio
import soundfile as sf
from demucs.apply import apply_model
from demucs.pretrained import get_model
from google.colab import drive

In [ ]:
import soundfile as sf
print(sf.__version__)

0.12.1


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


##!!!!! You need to add a shortcut from our CS 230 folder to your My Drive. You can do this by right clicking on the CS 230 folder and Organize --> Add Shortcut.

In [ ]:
zip_path = '/content/drive/MyDrive/CS 230/musdb18.zip'  # Path to your zip file
unzip_folder = '/content/musdb_unzipped'              # Where to unzip files

In [ ]:
# Unzipping the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_folder)

print("Dataset unzipped to:", unzip_folder)

Dataset unzipped to: /content/musdb_unzipped


In [ ]:
# Define the path to MUSDB test files
root = os.path.join(unzip_folder, "test")
if not os.path.exists(root):
    print(f"Directory not found: {root}")
else:
    print(f"Directory exists: {root}")
# Initialize Demucs model (we'll use the 'mdx_extra' pretrained model)
model = get_model('mdx_extra')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/e51eebcc-c1b80bdd.th" to /root/.cache/torch/hub/checkpoints/e51eebcc-c1b80bdd.th


Directory exists: /content/musdb_unzipped/test


100%|██████████| 160M/160M [00:01<00:00, 98.6MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/a1d90b5c-ae9d2452.th" to /root/.cache/torch/hub/checkpoints/a1d90b5c-ae9d2452.th
100%|██████████| 160M/160M [00:01<00:00, 129MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/5d2d6c55-db83574e.th" to /root/.cache/torch/hub/checkpoints/5d2d6c55-db83574e.th
100%|██████████| 160M/160M [00:01<00:00, 124MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/cfa93e08-61801ae1.th" to /root/.cache/torch/hub/checkpoints/cfa93e08-61801ae1.th
100%|██████████| 160M/160M [00:01<00:00, 145MB/s]


BagOfModels(
  (models): ModuleList(
    (0): HDemucs(
      (encoder): ModuleList(
        (0): HEncLayer(
          (conv): Conv2d(4, 48, kernel_size=(8, 1), stride=(4, 1), padding=(2, 0))
          (norm1): Identity()
          (rewrite): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1))
          (norm2): Identity()
          (dconv): DConv(
            (layers): ModuleList(
              (0): Sequential(
                (0): Conv1d(48, 12, kernel_size=(3,), stride=(1,), padding=(1,))
                (1): GroupNorm(1, 12, eps=1e-05, affine=True)
                (2): GELU(approximate='none')
                (3): Conv1d(12, 96, kernel_size=(1,), stride=(1,))
                (4): GroupNorm(1, 96, eps=1e-05, affine=True)
                (5): GLU(dim=1)
                (6): LayerScale()
              )
              (1): Sequential(
                (0): Conv1d(48, 12, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
                (1): GroupNorm(1, 12, eps=1e-05, affine=True

In [ ]:
# Define your file for separation
file = 'AM Contra - Heart Peripheral.stem.mp4'  # Specify your file here
input_path = os.path.join(root, file)
output_path = os.path.join(root, "demucs_separated", os.path.splitext(file)[0])
os.makedirs(output_path, exist_ok=True)

# Load audio file and move waveform to the appropriate device
waveform, sr = torchaudio.load(input_path)
waveform = waveform.to(device)

In [ ]:
# Separate sources
sources = apply_model(model, waveform.unsqueeze(0), device=device)

In [ ]:
# Remove the batch dimension
sources = sources.squeeze(0)  # Shape now becomes (4, 2, 9256960)

output_format = 'WAV'
subtype = 'PCM_16'
endian = 'FILE'

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Save each separated source to the output path
for i, source in enumerate(sources):
    stem_name = ["drums", "bass", "other", "vocals"][i]  # Adjust this if the model produces different stems
    output_file = os.path.join(output_path, f"{stem_name}.wav")

    # Ensure data is contiguous and in float32 format
    source = source.cpu().contiguous().float()
    print(f"Shape after converting to float32: {source.shape}")
    print(f"Saving {stem_name} to {output_file}, shape: {source.shape}, dtype: {source.dtype}")

    # Open a file descriptor with specific format parameters
    with sf.SoundFile(output_file, mode='w', samplerate=sr, channels=source.shape[0],
                      format=output_format, subtype=subtype, endian=endian) as f:
        f.write(source.T)  # Transpose to match (samples, channels) shape expected by soundfile

    # sf.write(output_file, source.squeeze(0).cpu().numpy(), sr, format="WAV")

print(f"Processed {file} and saved results to {output_path}")

Shape after converting to float32: torch.Size([2, 9256960])
Saving drums to /content/musdb_unzipped/test/demucs_separated/AM Contra - Heart Peripheral.stem/drums.wav, shape: torch.Size([2, 9256960]), dtype: torch.float32
Shape after converting to float32: torch.Size([2, 9256960])
Saving bass to /content/musdb_unzipped/test/demucs_separated/AM Contra - Heart Peripheral.stem/bass.wav, shape: torch.Size([2, 9256960]), dtype: torch.float32
Shape after converting to float32: torch.Size([2, 9256960])
Saving other to /content/musdb_unzipped/test/demucs_separated/AM Contra - Heart Peripheral.stem/other.wav, shape: torch.Size([2, 9256960]), dtype: torch.float32
Shape after converting to float32: torch.Size([2, 9256960])
Saving vocals to /content/musdb_unzipped/test/demucs_separated/AM Contra - Heart Peripheral.stem/vocals.wav, shape: torch.Size([2, 9256960]), dtype: torch.float32
Processed AM Contra - Heart Peripheral.stem.mp4 and saved results to /content/musdb_unzipped/test/demucs_separated/A